In [1]:
import pickle as pkl
import numpy as np
import pandas as pd
import sklearn
from twitter_nlp_toolkit import twit_module
from twitter_nlp_toolkit.file_fetcher import file_fetcher

Using TensorFlow backend.


In [2]:
downloader = file_fetcher.downloader(using_notebook=True)

downloader initialized


In [3]:
from IPython.display import clear_output, display

In [4]:
#Training data 
downloader.download_file("https://www.dropbox.com/s/5zr4e84x83vevbt/training.1600000.processed.noemoticon.csv.zip?dl=1","1_6_m_tweets.zip")

Downloading: 99% [81100800 / 81334274] bytes            


In [6]:
# Testing data
downloader.download_file('https://www.dropbox.com/s/440m6x07bjg6c0h/Tweets.zip?dl=1',"Tweets.zip")

In [7]:
# Pickled model
downloader.download_file("https://www.dropbox.com/s/owpldku3kk7aaqj/tweet_classifier_001.zip?dl=1","tweet_classifier_005.pkl")

Downloading: 99% [932806656 / 933152569] bytes            


We will use a dataset of general tweets (double-check where it's from) for training, and a dataset of hand-labeled customer feedback to airlines for validation



In [8]:
train_data = pd.read_csv("1_6_m_tweets.zip", encoding='latin-1',
                         names=['Labels', 'Index', 'Time', 'Query', 'Handle', 'Text'])

test_data = pd.read_csv('Tweets.zip', header=0, names=['Index', 'Sentiment', 'Sentiment_confidence',
                                                                'Negative_reason', 'Negative_reason_confidence',
                                                                'Airline', 'Airline_sentiment_gold', 'Handle',
                                                                'Negative_reason_gold', 'Retweet_count', 'Text',
                                                                'Tweet_coord', 'Time', 'Location', 'Timezone'])

In [9]:
Classifier = twit_module.SentimentAnalyzer(bow_param={}, lstm_param=None, glove_param=None)

In [10]:
Classifier.fit(train_data['Text'], train_data["Labels"] / 4)

ModuleNotFoundError: No module named 'en_core_web_sm'

In [22]:
test_data['Labels'] = (test_data['Sentiment'] == 'positive') * 2

print("Test Accuracy: %.3f" % sklearn.metrics.accuracy_score(test_data['Labels']/2, Classifier.predict(test_data['Text']).reshape(-1)))

Test Accuracy: 0.716


# TODO Update this

This has only been trained on a small amount of the data. Training on all of the data takes ~30 hours.

In [11]:
EnsembledClassifier = pkl.load(open('tweet_classifier_005.pkl', 'rb'))

UnpicklingError: A load persistent id instruction was encountered,
but no persistent_load function was specified.

In [27]:
print("Test Accuracy: %.3f" % sklearn.metrics.accuracy_score(test_data['Labels']/2, EnsembledClassifier.predict(test_data['Text']).reshape(-1)))

Test Accuracy: 0.643


In [28]:
EnsembledClassifier.predict(['I am happy', 'I am sad', 'I am cheerful', 'I am mad'])

array([1., 0., 1., 0.])